In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

*Loading the Data:*

**ACS 1 Year Estimates** - the American Community Survey (ACS) provides tract-level population estimates for the 2011-2015 period. ACS estimates were downloaded from the Census Bureau's FactFinder application and from the National Historic GIS website.

**FCC** - Tract-level data on household Internet connectivity is from the Federal Communications Commission's (FCC) Form 477 Census Tract Data on Internet Access Services (current as of June 30, 2016).


**IPUMS** - The Integrated Public Use Microdata Series (IPUMS-USA) provides user level socioeconomic data from as early as 1950-2016 organized by Public Use Microdata Area. Socioeconomic data includes touchpoints on: Rural / Urban Settings, Household Demographics, Gender, Age, Marriage, Race & Ethnicity, Education, Work, Income

**census_tract_converter** - code lookup to convert PUMA to census tract data

**Dropbox for Data** - https://www.dropbox.com/sh/v3viqffuphbpvee/AACm5N4_c7jGzTPxiYbeNo1sa?dl=0


In [11]:
#acs 1 year estimate
acs = pd.read_csv("ACS_16_5YR_B01003_with_all_states.csv", sep = ",", encoding = "latin-1", header = 1)

#fcc data
fcc = pd.read_csv("fbd_us_without_satellite_dec2016_v1.csv", sep=",", encoding = "latin-1")

#IPUMs data broken out by PUMA Codes with Data Dictionary (DDI)
#https://usa.ipums.org/usa-action/extract_requests/download, login: robert.y.deng@gmail.com // w209
ipums = pd.read_csv("usa_00005.csv", sep=",", encoding="latin-1")

#Puma to census tract converter, https://www.census.gov/geo/maps-data/data/centract_rel.html
ptc = pd.read_csv("2010_Census_Tract_to_2010_PUMA.txt", sep=',', encoding='latin-1')

In [12]:
print("ACS Shape", acs.shape, "\nFCC Shape", fcc.shape, 
      "\nIPUMS Shape", ipums.shape, "\nPUMA Code Converter", ptc.shape)

ACS Shape (74001, 5) 
FCC Shape (25738665, 17) 
IPUMS Shape (3156487, 24) 
PUMA Code Converter (74091, 4)


In [13]:
ipums.sample(5)

,YEAR,DATANUM,SERIAL,HHWT,PUMA,GQ,FARM,PERNUM,PERWT,FAMSIZE,...,YRSUSA1,SPEAKENG,HCOVANY,GRADEATT,GRADEATTD,DEGFIELD,DEGFIELDD,EMPSTAT,EMPSTATD,INCWAGE
1473004,2016,1,639734,73,3203,1,1,1,73,2,...,31,5,2,0,0,0,0,1,10,38000
1869895,2016,1,814630,47,500,1,1,1,47,2,...,0,3,2,0,0,52,5200,1,10,4800
526577,2016,1,215268,90,3750,1,1,1,90,5,...,30,6,2,0,0,0,0,1,10,18000
2017604,2016,1,878702,41,1204,1,1,2,55,2,...,0,4,2,0,0,0,0,3,30,0
2879794,2016,1,1260057,21,1400,1,1,2,47,3,...,0,3,2,0,0,0,0,3,30,0


Cleaning the IPUMS data, there are 3 steps:
    1. Dummify ordinal variables and remove the original ordinal columns
    2. Groupby Serial number, which is user level identifier data
        - Sum and average for specific metrics separately and then merge
    3. Groupby PUMA code 
        - Sum and average for specific metrics separately and then merge

In [14]:
#dummify everything with an ordinal variable:
#race, farm, Citizen, speak english, healthcare coverage, grade attended, employment
ipums_cleaned = pd.concat([ipums.drop(ipums[["YEAR", "DATANUM", "HHWT", "GQ", "PERNUM", "PERWT", "YRIMMIG",
                                            "RACE", "FARM", "CITIZEN", "SPEAKENG", "HCOVANY", 
                                            "GRADEATT", "DEGFIELD", "EMPSTAT", 
                                            "GRADEATTD", "RACED", 
                                            "DEGFIELDD", "EMPSTATD"]], axis = 1),
                          pd.get_dummies(ipums["RACE"], prefix="RACE"),
                          pd.get_dummies(ipums["FARM"], prefix="FARM"),
                          pd.get_dummies(ipums["CITIZEN"], prefix="CITIZEN"), 
                          pd.get_dummies(ipums["SPEAKENG"], prefix="SPEAKS_ENGLISH"), 
                          pd.get_dummies(ipums["HCOVANY"], prefix="HEALTH_CARE_COVERAGE"), 
                          pd.get_dummies(ipums["GRADEATT"], prefix = "EDUCATION"), 
                          #pd.get_dummies(ipums["DEGFIELD"], prefix="FIELD_OF_DEGREE"), Just TMI, so just remove it
                          pd.get_dummies(ipums["EMPSTAT"], prefix="EMPLOYMENT")], axis = 1)


ipums_cleaned.head(n=50)

#Famsize - average
#YRSUSA - median, or non-zero count
#IncWage - sum
#everything else just sum

,SERIAL,PUMA,FAMSIZE,YRSUSA1,INCWAGE,RACE_1,RACE_2,RACE_3,RACE_4,RACE_5,...,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,EMPLOYMENT_0,EMPLOYMENT_1,EMPLOYMENT_2,EMPLOYMENT_3
0,1,700,2,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,700,2,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,900,1,0,27300,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,302,4,0,65000,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,3,302,4,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,3,302,4,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,3,302,4,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7,4,600,2,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,4,600,2,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,5,2500,2,0,78000,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
print("{:.2f} Total columns for IPUMS Cleaned".format(len(ipums_cleaned.columns),), 
      "\n\n", ipums_cleaned.columns)

40.00 Total columns for IPUMS Cleaned 

 Index(['SERIAL', 'PUMA', 'FAMSIZE', 'YRSUSA1', 'INCWAGE', 'RACE_1', 'RACE_2',
       'RACE_3', 'RACE_4', 'RACE_5', 'RACE_6', 'RACE_7', 'RACE_8', 'RACE_9',
       'FARM_1', 'FARM_2', 'CITIZEN_0', 'CITIZEN_1', 'CITIZEN_2', 'CITIZEN_3',
       'SPEAKS_ENGLISH_0', 'SPEAKS_ENGLISH_1', 'SPEAKS_ENGLISH_3',
       'SPEAKS_ENGLISH_4', 'SPEAKS_ENGLISH_5', 'SPEAKS_ENGLISH_6',
       'HEALTH_CARE_COVERAGE_1', 'HEALTH_CARE_COVERAGE_2', 'EDUCATION_0',
       'EDUCATION_1', 'EDUCATION_2', 'EDUCATION_3', 'EDUCATION_4',
       'EDUCATION_5', 'EDUCATION_6', 'EDUCATION_7', 'EMPLOYMENT_0',
       'EMPLOYMENT_1', 'EMPLOYMENT_2', 'EMPLOYMENT_3'],
      dtype='object')


In [16]:
#First groupby individual user level data in SERIAL, average for family size and YRS in the USA
ipums_groupby_average = ipums_cleaned.ix[:,0:4].groupby(["SERIAL"], as_index = False).mean()
ipums_groupby_average.head(n=5)

,SERIAL,PUMA,FAMSIZE,YRSUSA1
0,1,700.0,2.0,0.0
1,2,900.0,1.0,0.0
2,3,302.0,4.0,0.0
3,4,600.0,2.0,0.0
4,5,2500.0,2.0,0.0


In [17]:
#Groupby individual user level data in SERIAL, sum for every other metric
ix_to_sum = list([0]) + list(range(5,len(ipums_cleaned.columns)))
ipums_groupby_sum = ipums_cleaned.ix[:,ix_to_sum].groupby(["SERIAL"], as_index = False).sum()
ipums_groupby_sum.head()

,SERIAL,RACE_1,RACE_2,RACE_3,RACE_4,RACE_5,RACE_6,RACE_7,RACE_8,RACE_9,...,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,EMPLOYMENT_0,EMPLOYMENT_1,EMPLOYMENT_2,EMPLOYMENT_3
0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0
3,4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [18]:
#Merge sum and average metrics
ipums_cleaned_2 = ipums_groupby_average.merge(ipums_groupby_sum, left_on = "SERIAL", right_on = "SERIAL", how = "left")
ipums_cleaned_2.head()

,SERIAL,PUMA,FAMSIZE,YRSUSA1,RACE_1,RACE_2,RACE_3,RACE_4,RACE_5,RACE_6,...,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,EMPLOYMENT_0,EMPLOYMENT_1,EMPLOYMENT_2,EMPLOYMENT_3
0,1,700.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,2,900.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,302.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0
3,4,600.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,5,2500.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [19]:
#Now groupby PUMA, sum and average in the same way and rejoin
ipums_groupby_average_2 = ipums_cleaned_2.ix[:,1:4].groupby(["PUMA"], as_index = False).mean()
ix_to_sum_2 = list([1]) + list(range(5,39))
ipums_groupby_sum_2 = ipums_cleaned_2.ix[:,ix_to_sum_2].groupby(["PUMA"], as_index = False).sum()

#Final left join by PUMA, ready to merge with census data
ipums_master = ipums_groupby_average_2.merge(ipums_groupby_sum_2, left_on = "PUMA", right_on = "PUMA", how = "left")
ipums_master.head()

,PUMA,FAMSIZE,YRSUSA1,RACE_2,RACE_3,RACE_4,RACE_5,RACE_6,RACE_7,RACE_8,...,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,EMPLOYMENT_0,EMPLOYMENT_1,EMPLOYMENT_2,EMPLOYMENT_3
0,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,1406.0,748.0,1872.0,...,790.0,3175.0,3234.0,3370.0,3178.0,461.0,12405.0,29746.0,1633.0,23550.0
1,101.0,1.969199,2.744565,2084.0,165.0,326.0,53.0,741.0,579.0,616.0,...,174.0,830.0,820.0,797.0,2108.0,525.0,3126.0,8825.0,601.0,6638.0
2,102.0,2.222179,3.388744,2520.0,197.0,382.0,52.0,703.0,732.0,628.0,...,222.0,916.0,932.0,994.0,1002.0,308.0,3652.0,10447.0,507.0,6311.0
3,103.0,2.096533,3.106207,1754.0,191.0,238.0,30.0,545.0,631.0,534.0,...,155.0,648.0,679.0,713.0,1207.0,304.0,2666.0,8043.0,412.0,4603.0
4,104.0,2.182327,3.108139,2603.0,63.0,101.0,10.0,236.0,817.0,286.0,...,154.0,545.0,517.0,571.0,593.0,150.0,2133.0,5393.0,394.0,3757.0


In [20]:
ipums_merged_ptc = ipums_master.merge(ptc, left_on = "PUMA", right_on = "PUMA5CE", how = "left")
ipums_merged_ptc.head()

,PUMA,FAMSIZE,YRSUSA1,RACE_2,RACE_3,RACE_4,RACE_5,RACE_6,RACE_7,RACE_8,...,EDUCATION_6,EDUCATION_7,EMPLOYMENT_0,EMPLOYMENT_1,EMPLOYMENT_2,EMPLOYMENT_3,STATEFP,COUNTYFP,TRACTCE,PUMA5CE
0,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,1406.0,748.0,1872.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,1,33,20100,100
1,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,1406.0,748.0,1872.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,1,33,20200,100
2,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,1406.0,748.0,1872.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,1,33,20300,100
3,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,1406.0,748.0,1872.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,1,33,20400,100
4,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,1406.0,748.0,1872.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,1,33,20500,100


In [25]:
acs["Census_Tract"] = (acs["Geography"].str.replace("Census Tract ", "").str.split(",").str[0].astype(float)*100).astype(int)
acs_to_merge = acs[["Census_Tract", "Margin of Error; Total", "Estimate; Total"]]
acs_to_merge.head()

,Census_Tract,Margin of Error; Total,Estimate; Total
0,20100,231,2010
1,20200,281,2196
2,20300,355,3136
3,20400,464,4563
4,20500,655,10529


In [26]:
acs_ipums = acs_to_merge.merge(ipums_merged_ptc, left_on = "Census_Tract", right_on = "TRACTCE", how = "left")
acs_ipums.head(n=10)

,Census_Tract,Margin of Error; Total,Estimate; Total,PUMA,FAMSIZE,YRSUSA1,RACE_2,RACE_3,RACE_4,RACE_5,...,EDUCATION_6,EDUCATION_7,EMPLOYMENT_0,EMPLOYMENT_1,EMPLOYMENT_2,EMPLOYMENT_3,STATEFP,COUNTYFP,TRACTCE,PUMA5CE
0,20100,231,2010,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,1.0,33.0,20100.0,100.0
1,20100,231,2010,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,21.0,83.0,20100.0,100.0
2,20100,231,2010,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,27.0,27.0,20100.0,100.0
3,20100,231,2010,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,27.0,119.0,20100.0,100.0
4,20100,231,2010,100.0,2.161538,1.197337,2453.0,3838.0,132.0,160.0,...,3178.0,461.0,12405.0,29746.0,1633.0,23550.0,50.0,13.0,20100.0,100.0
5,20100,231,2010,101.0,1.969199,2.744565,2084.0,165.0,326.0,53.0,...,2108.0,525.0,3126.0,8825.0,601.0,6638.0,18.0,89.0,20100.0,101.0
6,20100,231,2010,102.0,2.222179,3.388744,2520.0,197.0,382.0,52.0,...,1002.0,308.0,3652.0,10447.0,507.0,6311.0,9.0,1.0,20100.0,102.0
7,20100,231,2010,104.0,2.182327,3.108139,2603.0,63.0,101.0,10.0,...,593.0,150.0,2133.0,5393.0,394.0,3757.0,17.0,85.0,20100.0,104.0
8,20100,231,2010,105.0,2.130796,3.428072,1045.0,44.0,305.0,31.0,...,599.0,216.0,1640.0,4994.0,224.0,2868.0,17.0,161.0,20100.0,105.0
9,20100,231,2010,200.0,2.122236,1.133435,2918.0,2623.0,144.0,184.0,...,3055.0,509.0,11132.0,28062.0,1588.0,22742.0,15.0,1.0,20100.0,200.0


In [27]:
fcc["Census_Tract"] = fcc["BlockCode"].astype(str).str[5:11]
fcc.head()

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp,Census_Tract
0,1,25422,18547471,"Gans Communications, L.P. d/b/a MetroCast Comm...",Metrocast,Harron Communications LP,130591,Harron Communications LP,VA,510330301001048,42,1,105.0,10.0,1,150.0,10.0,030100
1,2,25422,18547471,"Gans Communications, L.P. d/b/a MetroCast Comm...",Metrocast,Harron Communications LP,130591,Harron Communications LP,VA,510330301001054,42,1,105.0,10.0,1,150.0,10.0,030100
2,3,25422,18547471,"Gans Communications, L.P. d/b/a MetroCast Comm...",Metrocast,Harron Communications LP,130591,Harron Communications LP,VA,510330301001067,42,1,105.0,10.0,1,150.0,10.0,030100
3,4,25422,18547471,"Gans Communications, L.P. d/b/a MetroCast Comm...",Metrocast,Harron Communications LP,130591,Harron Communications LP,VA,510330301001068,42,1,105.0,10.0,1,150.0,10.0,030100
4,5,25422,18547471,"Gans Communications, L.P. d/b/a MetroCast Comm...",Metrocast,Harron Communications LP,130591,Harron Communications LP,VA,510330301001069,42,1,105.0,10.0,1,150.0,10.0,030100


In [28]:
fcc_providers = pd.DataFrame(fcc.groupby('Census_Tract').Provider_Id.nunique()).reset_index()
#fcc_providers.rename(index=str, columns={"Census_Tract" : "Census_Tract", "Provider_Counts" : "Provider_Counts"})
fcc_providers.head()

,Census_Tract,Provider_Id
0,000100,511
1,000101,134
2,000102,120
3,000103,57
4,000104,26


In [29]:
master_merged = acs_ipums.merge(fcc_providers, left_on = "Census_Tract", right_on = "Census_Tract", how = "left")

In [30]:
acs_ipums.to_csv("Master_Data.csv")